This document covers the cleaning of the dataset.



#Install and import libraries

Installation of Spark in Google Colab (Code extracted from: https://www.pauldesalvo.com/how-to-install-spark-on-google-colab/)

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()

spark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

Import other needed libraries

In [ ]:
import pyspark.sql.functions as F
import seaborn as sns
import datetime

# Custom methods definition

In [ ]:
#Method to extract percentiles of a given variable
def show_quantiles(df, variable_to_analyze):

  return df.agg(F.expr('percentile('+variable_to_analyze+', 0.00)').alias('%0'),
                F.expr('percentile('+variable_to_analyze+', 0.01)').alias('%1'),
                F.expr('percentile('+variable_to_analyze+', 0.02)').alias('%2'),
                F.expr('percentile('+variable_to_analyze+', 0.03)').alias('%3'),
                F.expr('percentile('+variable_to_analyze+', 0.04)').alias('%4'),
                F.expr('percentile('+variable_to_analyze+', 0.05)').alias('%5'),

                F.expr('percentile('+variable_to_analyze+', 0.10)').alias('%10'),
                F.expr('percentile('+variable_to_analyze+', 0.15)').alias('%15'),
                F.expr('percentile('+variable_to_analyze+', 0.20)').alias('%20'),
                F.expr('percentile('+variable_to_analyze+', 0.25)').alias('%25'),
                F.expr('percentile('+variable_to_analyze+', 0.30)').alias('%30'),
                F.expr('percentile('+variable_to_analyze+', 0.35)').alias('%35'),
                F.expr('percentile('+variable_to_analyze+', 0.40)').alias('%40'),
                F.expr('percentile('+variable_to_analyze+', 0.45)').alias('%45'),
                F.expr('percentile('+variable_to_analyze+', 0.50)').alias('%50'),
                F.expr('percentile('+variable_to_analyze+', 0.55)').alias('%55'),
                F.expr('percentile('+variable_to_analyze+', 0.60)').alias('%60'),
                F.expr('percentile('+variable_to_analyze+', 0.65)').alias('%65'),
                F.expr('percentile('+variable_to_analyze+', 0.70)').alias('%70'),
                F.expr('percentile('+variable_to_analyze+', 0.75)').alias('%75'),
                F.expr('percentile('+variable_to_analyze+', 0.80)').alias('%80'),
                F.expr('percentile('+variable_to_analyze+', 0.85)').alias('%85'),
                F.expr('percentile('+variable_to_analyze+', 0.90)').alias('%90'),

                F.expr('percentile('+variable_to_analyze+', 0.95)').alias('%95'),
                F.expr('percentile('+variable_to_analyze+', 0.96)').alias('%96'),
                F.expr('percentile('+variable_to_analyze+', 0.97)').alias('%97'),
                F.expr('percentile('+variable_to_analyze+', 0.98)').alias('%98'),
                F.expr('percentile('+variable_to_analyze+', 0.99)').alias('%99'),
                F.expr('percentile('+variable_to_analyze+', 1.00)').alias('%100')
              )

In [ ]:
#Method to extract percentiles of a given variable (with more detail at the tails of the distribution)
def show_quantiles_with_tail_detail(df, variable_to_analyze):
  return df.agg(F.expr('percentile('+variable_to_analyze+', 0.00)').alias('%0'),
                
                F.expr('percentile('+variable_to_analyze+', 0.001)').alias('%0.1'),
                F.expr('percentile('+variable_to_analyze+', 0.005)').alias('%0.5'),

                F.expr('percentile('+variable_to_analyze+', 0.01)').alias('%1'),
                F.expr('percentile('+variable_to_analyze+', 0.02)').alias('%2'),
                F.expr('percentile('+variable_to_analyze+', 0.03)').alias('%3'),
                F.expr('percentile('+variable_to_analyze+', 0.04)').alias('%4'),
                F.expr('percentile('+variable_to_analyze+', 0.05)').alias('%5'),

                F.expr('percentile('+variable_to_analyze+', 0.10)').alias('%10'),
                F.expr('percentile('+variable_to_analyze+', 0.15)').alias('%15'),
                F.expr('percentile('+variable_to_analyze+', 0.20)').alias('%20'),
                F.expr('percentile('+variable_to_analyze+', 0.25)').alias('%25'),
                F.expr('percentile('+variable_to_analyze+', 0.30)').alias('%30'),
                F.expr('percentile('+variable_to_analyze+', 0.35)').alias('%35'),
                F.expr('percentile('+variable_to_analyze+', 0.40)').alias('%40'),
                F.expr('percentile('+variable_to_analyze+', 0.45)').alias('%45'),
                F.expr('percentile('+variable_to_analyze+', 0.50)').alias('%50'),
                F.expr('percentile('+variable_to_analyze+', 0.55)').alias('%55'),
                F.expr('percentile('+variable_to_analyze+', 0.60)').alias('%60'),
                F.expr('percentile('+variable_to_analyze+', 0.65)').alias('%65'),
                F.expr('percentile('+variable_to_analyze+', 0.70)').alias('%70'),
                F.expr('percentile('+variable_to_analyze+', 0.75)').alias('%75'),
                F.expr('percentile('+variable_to_analyze+', 0.80)').alias('%80'),
                F.expr('percentile('+variable_to_analyze+', 0.85)').alias('%85'),
                F.expr('percentile('+variable_to_analyze+', 0.90)').alias('%90'),

                F.expr('percentile('+variable_to_analyze+', 0.95)').alias('%95'),
                F.expr('percentile('+variable_to_analyze+', 0.96)').alias('%96'),
                F.expr('percentile('+variable_to_analyze+', 0.97)').alias('%97'),
                F.expr('percentile('+variable_to_analyze+', 0.98)').alias('%98'),
                F.expr('percentile('+variable_to_analyze+', 0.99)').alias('%99'),

                F.expr('percentile('+variable_to_analyze+', 0.995)').alias('%99.5'),
                F.expr('percentile('+variable_to_analyze+', 0.999)').alias('%99.9'),

                F.expr('percentile('+variable_to_analyze+', 1.00)').alias('%100')
                )

# Read data and clean dates

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
work_folder = '/content/drive/MyDrive/nyc_taxis/'

Download NYC data (Data downloaded from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

In [ ]:
!wget --continue https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-03.parquet -O $work_folder'yellow_tripdata_2017-03.parquet'

--2022-06-20 22:17:38--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-03.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.106.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.106.77|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
!wget --continue https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-06.parquet -O $work_folder'yellow_tripdata_2017-06.parquet


/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
!wget --continue https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2017-11.parquet -O $work_folder'yellow_tripdata_2017-11.parquet

/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
nyc_yellow_tripdata_2017_03 = spark.read.parquet(work_folder+"yellow_tripdata_2017-03.parquet")

In [ ]:
nyc_yellow_tripdata_2017_03.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2017-03-01 00:38:16|  2017-03-01 00:59:21|              1|         10.5|         1|                 N|         231|          42|           1|       30.5|  0.5|    0.5|       6.

In [ ]:
nyc_yellow_tripdata_2017_06 = spark.read.parquet(work_folder+"yellow_tripdata_2017-06.parquet")

In [ ]:
nyc_yellow_tripdata_2017_06.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2017-06-01 00:02:36|  2017-06-01 00:10:02|              1|          1.8|         1|                 N|         161|         263|           1|        8.0|  0.5|    0.5|      1.8

In [ ]:
nyc_yellow_tripdata_2017_11 = spark.read.parquet(work_folder+"yellow_tripdata_2017-11.parquet")

In [ ]:
nyc_yellow_tripdata_2017_11.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2017-11-01 00:01:48|  2017-11-01 00:03:47|              1|          0.4|         1|                 N|         151|         151|           2|        3.5|  0.5|    0.5|       0.

## Clean dates before merging datasets

### March 2017

Before we merge the three datasets together (March, June and November), we are going to make sure that the pick-up and drop-off dates are really in the months that they should be

In [ ]:
tpep_datetime_min = datetime.datetime(2017, 3, 1)
tpep_datetime_max = datetime.datetime(2017, 4, 1)

For March, the drop-off dates seem to contain strange values (at night, during the first day of April). We focus on analyzing those cases next.

In [ ]:
nyc_yellow_tripdata_2017_03.select(F.min("tpep_pickup_datetime"), F.max("tpep_pickup_datetime"),
                                   F.min("tpep_dropoff_datetime"), F.max("tpep_dropoff_datetime")).show()

+-------------------------+-------------------------+--------------------------+--------------------------+
|min(tpep_pickup_datetime)|max(tpep_pickup_datetime)|min(tpep_dropoff_datetime)|max(tpep_dropoff_datetime)|
+-------------------------+-------------------------+--------------------------+--------------------------+
|      2017-03-01 00:00:00|      2017-03-31 23:59:59|       2017-03-01 00:00:00|       2017-04-01 23:45:44|
+-------------------------+-------------------------+--------------------------+--------------------------+



There are some cases in which some trips have lasted for almost a whole day according to these variables. If we look at the trip distance, this does not make sense. Probably, the date columns have just been inverted and shifted one day. We next count the number of records to understand if this is happening too often.

In [ ]:
wrong_drop_off_times = nyc_yellow_tripdata_2017_03.filter(F.col("tpep_dropoff_datetime") >= tpep_datetime_max).sort(F.desc("tpep_dropoff_datetime"))
wrong_drop_off_times.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2017-03-31 23:57:03|  2017-04-01 23:45:44|              1|          6.5|         1|                 N|         233|          82|           2|       28.0|  0.5|    0.5|       0.

There are not many records with this problem, so we can directly drop these rows before merging the datasets.

In [ ]:
wrong_drop_off_times.count()

5273

In [ ]:
print(nyc_yellow_tripdata_2017_03.count())
nyc_yellow_tripdata_2017_03 = nyc_yellow_tripdata_2017_03.filter(F.col("tpep_dropoff_datetime") < tpep_datetime_max)
print(nyc_yellow_tripdata_2017_03.count())

10295441
10290168


### June 2017

In [ ]:
tpep_datetime_min = datetime.datetime(2017, 6, 1)
tpep_datetime_max = datetime.datetime(2017, 7, 1)

In [ ]:
nyc_yellow_tripdata_2017_06.select(F.min("tpep_pickup_datetime"), F.max("tpep_pickup_datetime"),
                                   F.min("tpep_dropoff_datetime"), F.max("tpep_dropoff_datetime")).show()

+-------------------------+-------------------------+--------------------------+--------------------------+
|min(tpep_pickup_datetime)|max(tpep_pickup_datetime)|min(tpep_dropoff_datetime)|max(tpep_dropoff_datetime)|
+-------------------------+-------------------------+--------------------------+--------------------------+
|      2017-06-01 00:00:00|      2017-06-30 23:59:58|       2017-06-01 00:00:00|       2017-07-01 23:53:05|
+-------------------------+-------------------------+--------------------------+--------------------------+



We can see the same problem ocurring in June, with trips lasting a whole day according to these variables. 

In [ ]:
wrong_drop_off_times = nyc_yellow_tripdata_2017_06.filter(F.col("tpep_dropoff_datetime") >= tpep_datetime_max).sort(F.desc("tpep_dropoff_datetime"))
wrong_drop_off_times.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2017-06-30 23:57:37|  2017-07-01 23:53:05|              1|         3.99|         1|                 N|          79|         262|           1|       18.0|  0.5|    0.5|       0.

Since there are just a few cases, we drop them from the dataset

In [ ]:
wrong_drop_off_times.count()

3713

In [ ]:
print(nyc_yellow_tripdata_2017_06.count())
nyc_yellow_tripdata_2017_06 = nyc_yellow_tripdata_2017_06.filter(F.col("tpep_dropoff_datetime") < tpep_datetime_max)
print(nyc_yellow_tripdata_2017_06.count())

9656993
9653280


### November 2017

In [ ]:
tpep_datetime_min = datetime.datetime(2017, 11, 1)
tpep_datetime_max = datetime.datetime(2017, 12, 1)

In this case, there are dates that are clearly wrong in the dataset (e.g., 2001, 2041).

In [ ]:
nyc_yellow_tripdata_2017_11.select(F.min("tpep_pickup_datetime"), F.max("tpep_pickup_datetime"),
                                   F.min("tpep_dropoff_datetime"), F.max("tpep_dropoff_datetime")).show()

+-------------------------+-------------------------+--------------------------+--------------------------+
|min(tpep_pickup_datetime)|max(tpep_pickup_datetime)|min(tpep_dropoff_datetime)|max(tpep_dropoff_datetime)|
+-------------------------+-------------------------+--------------------------+--------------------------+
|      2001-01-01 00:04:13|      2041-11-15 02:57:16|       2001-01-01 00:04:51|       2041-11-15 03:12:19|
+-------------------------+-------------------------+--------------------------+--------------------------+



Since there aren't many of these cases, we just remove them cases before merging the three datasets together:

In [ ]:
print(nyc_yellow_tripdata_2017_11.filter(F.col("tpep_pickup_datetime") < tpep_datetime_min).count())
print(nyc_yellow_tripdata_2017_11.filter(F.col("tpep_dropoff_datetime") < tpep_datetime_min).count())
print(nyc_yellow_tripdata_2017_11.filter(F.col("tpep_pickup_datetime") >= tpep_datetime_max).count())
print(nyc_yellow_tripdata_2017_11.filter(F.col("tpep_dropoff_datetime") >= tpep_datetime_max).count())

236
104
84
4048


In [ ]:
print(nyc_yellow_tripdata_2017_11.count())
nyc_yellow_tripdata_2017_11 = nyc_yellow_tripdata_2017_11.filter(F.col("tpep_dropoff_datetime") >= tpep_datetime_min)
nyc_yellow_tripdata_2017_11 = nyc_yellow_tripdata_2017_11.filter(F.col("tpep_dropoff_datetime") < tpep_datetime_max)
print(nyc_yellow_tripdata_2017_11.count())

9284803
9280651


##Clean dates after merging datasets

In [ ]:
print(nyc_yellow_tripdata_2017_03.count())
print(nyc_yellow_tripdata_2017_06.count())
print(nyc_yellow_tripdata_2017_11.count())

10290168
9653280
9280651


We merge the datasets together:

In [ ]:
nyc_yellow_tripdata = nyc_yellow_tripdata_2017_03.union(nyc_yellow_tripdata_2017_06).union(nyc_yellow_tripdata_2017_11)

We check that the final dates are in the expected ranges:

In [ ]:
tpep_datetime_min = datetime.datetime(2017, 3, 1)
tpep_datetime_max = datetime.datetime(2017, 12, 1)

In [ ]:
nyc_yellow_tripdata.select(F.min("tpep_pickup_datetime"), F.max("tpep_pickup_datetime"),
                           F.min("tpep_dropoff_datetime"), F.max("tpep_dropoff_datetime")).show()

+-------------------------+-------------------------+--------------------------+--------------------------+
|min(tpep_pickup_datetime)|max(tpep_pickup_datetime)|min(tpep_dropoff_datetime)|max(tpep_dropoff_datetime)|
+-------------------------+-------------------------+--------------------------+--------------------------+
|      2017-03-01 00:00:00|      2017-11-30 23:59:05|       2017-03-01 00:00:00|       2017-11-30 23:59:59|
+-------------------------+-------------------------+--------------------------+--------------------------+



We now check if any pick-off dates happen in time before the corresponding pick-up date, or if the two dates are exactly the same.

In [ ]:
print(nyc_yellow_tripdata.filter(F.col("tpep_pickup_datetime") > F.col("tpep_dropoff_datetime")).count())
print(nyc_yellow_tripdata.filter(F.col("tpep_pickup_datetime") == F.col("tpep_dropoff_datetime")).count())

1419
28051


Since these cases do not make any sense, and they do not happen too often, so we remove them from the dataset.

In [ ]:
print(nyc_yellow_tripdata.count())
nyc_yellow_tripdata = nyc_yellow_tripdata.filter(F.col("tpep_pickup_datetime") < F.col("tpep_dropoff_datetime"))
print(nyc_yellow_tripdata.count())

29224099
29194629


# Feature formatting & engineering

We have a look at the schema to understand the type of variables that we have.
Two of them are timestamps (tpep_pickup_datetime, tpep_dropoff_datetime), and another one is of type string (store_and_fwd_flag). The other variables are numbers.

In [ ]:
nyc_yellow_tripdata.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: integer (nullable = true)
 |-- airport_fee: integer (nullable = true)



## Store_and_fwd_flag

In [ ]:
nyc_yellow_tripdata.groupby('store_and_fwd_flag').count().show()

+------------------+--------+
|store_and_fwd_flag|   count|
+------------------+--------+
|                 Y|  123154|
|                 N|29071475|
+------------------+--------+



This variable can be represented as 1/0 instead of Y/N, so we make the conversion:

In [ ]:
nyc_yellow_tripdata = nyc_yellow_tripdata.withColumn("store_and_fwd_flag",F.when(nyc_yellow_tripdata["store_and_fwd_flag"] == "Y", 1).otherwise(0))

In [ ]:
nyc_yellow_tripdata.groupby('store_and_fwd_flag').count().show()

+------------------+--------+
|store_and_fwd_flag|   count|
+------------------+--------+
|                 1|  123154|
|                 0|29071475|
+------------------+--------+



## Calculation of meter engagement time

Given the drop off and pick up times, we can calculate the total meter engagement time:

In [ ]:
nyc_yellow_tripdata = nyc_yellow_tripdata.withColumn("meter_engagement_time_minutes",F.round((F.unix_timestamp("tpep_dropoff_datetime")-F.unix_timestamp("tpep_pickup_datetime"))/60.0, 2))

# Data cleaning (excluding dates, which we already cleaned earlier)

We remove full duplicates from the dataset and cache the dataset and this point (to improve the performance of the next jobs, since recomputing the in-memory columnar representation of the underlying table during the next cleaning steps would be expensive):

In [ ]:
print(nyc_yellow_tripdata.count())
nyc_yellow_tripdata = nyc_yellow_tripdata.dropDuplicates().cache()
print(nyc_yellow_tripdata.count())

29194629
29194600


We check the general metrics of the dataset to understand the distribution of the data and see if there are any outliers or values that do not make sense.

We can see from the data distribution that some variables have clear outliers (passager_count = 192, fare_amount = 630461), and there are values that do not make sense (negative values for fare_amount, extra, etc.). 

In [ ]:
nyc_yellow_tripdata.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+--------------------+------------------+------------------+---------------------+------------------+--------------------+-----------+-----------------------------+
|summary|          VendorID|   passenger_count|     trip_distance|        RatecodeID|  store_and_fwd_flag|      PULocationID|      DOLocationID|       payment_type|       fare_amount|             extra|             mta_tax|        tip_amount|      tolls_amount|improvement_surcharge|      total_amount|congestion_surcharge|airport_fee|meter_engagement_time_minutes|
+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+--------------------+------------------+------------------+-------

In order to clean the variables, we will substitute the original values with null values in the cases that the original values do not make sense. We could also take other approaches, such as dropping all rows that contain incorrect/outlier values for any of the variables, but we can still extract meaningful information from the other variables in these cases, so this last procedure has been disregarded.

We create a new variable which will contain the clean dataset:

In [ ]:
nyc_yellow_tripdata_clean = nyc_yellow_tripdata

We can now start to clean each variable individually

## Congestion_Surcharge & Airport_fee

From the schema above, we can see that two of the variables do not contain any values, since their count is 0. We can safely delete these variables, as they are useless if they are completely null.

In [ ]:
nyc_yellow_tripdata_clean = nyc_yellow_tripdata.drop("congestion_surcharge","airport_fee")

## Meter_Engagement_Time_Minutes

In [ ]:
variable_to_clean = "meter_engagement_time_minutes"

We can observe from the data distribution below that meter_engagement_time_minutes has clear outliers: values that are way too close to zero, such as 0.02 minutes, or values that are way too high, such as 14407 minutes.

Depending on the problem, it could be interesting to analyze the nature of these outliers (for example, the data collection procedure could be misbehaving in some cases, and it would be useful to identify and correct those cases). Other times, we could be solely interested in outlier detection (for example, to detect fraudulent bank transactions).

In this case, since the final goal is to build a machine learning model that behaves well in the common cases, we can remove outliers.

In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+----+----+----+----+----+---+---+----+----+---+----+----+----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+--------+
|  %0|  %1|  %2|  %3|  %4| %5|%10| %15| %20|%25| %30| %35| %40|  %45|  %50|  %55|  %60| %65|  %70|  %75|  %80|  %85|  %90|  %95|  %96|  %97| %98|  %99|    %100|
+----+----+----+----+----+---+---+----+----+---+----+----+----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+--------+
|0.02|1.27|1.97|2.38|2.72|3.0|4.1|5.02|5.87|6.7|7.55|8.42|9.32|10.28|11.32|12.43|13.68|15.1|16.73|18.68|21.12|24.35|29.13|38.43|41.82|46.37|53.0|64.53|14407.65|
+----+----+----+----+----+---+---+----+----+---+----+----+----+-----+-----+-----+-----+----+-----+-----+-----+-----+-----+-----+-----+-----+----+-----+--------+



We remove the lowest (below the 1% percentile) and highest tail (above the 99% percentile) of the distribution

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.01)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) > nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.99)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
28618495


In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+----+-----+-----+----+-----+----+-----+-----+-----+-----+-----+-----+
|  %0|  %1|  %2|  %3|  %4|  %5| %10| %15| %20| %25| %30| %35| %40| %45|  %50|  %55|  %60| %65|  %70|  %75| %80|  %85| %90|  %95|  %96|  %97|  %98|  %99| %100|
+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+----+-----+-----+----+-----+----+-----+-----+-----+-----+-----+-----+
|1.27|1.93|2.37|2.68|2.97|3.22|4.25|5.13|5.97|6.78|7.62|8.47|9.35|10.3|11.32|12.42|13.63|15.0|16.58|18.47|20.8|23.83|28.2|36.08|38.72|42.08|46.65|53.25|64.72|
+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+----+-----+-----+----+-----+----+-----+-----+-----+-----+-----+-----+



## Passenger_count

According to the data dictionary, this is a driver-entered value, so there is a high probability that it contains wrong values because of human errors, since the dataset is quite large.

In [ ]:
variable_to_clean = "passenger_count"

There are some taxi trips with zero passangers. We can remove this, since if there are no passangers, there shouldn't be any passanger tips, and that's what we want to predict in the end.

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) <= 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29143388


Below we can see all of the values that this variable has. The highest one (192) does not make any sense and we can remove it. The other high values (e.g., 7, 8, 9, etc.) could make sense for large taxies, and there are not many cases with those variables, so we can leave them there.

In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().orderBy(variable_to_clean).show()

+---------------+--------+
|passenger_count|   count|
+---------------+--------+
|           null|   51212|
|              1|20929842|
|              2| 4214033|
|              3| 1203562|
|              4|  557849|
|              5| 1387397|
|              6|  850462|
|              7|      88|
|              8|      88|
|              9|      66|
|            192|       1|
+---------------+--------+



We remove the highest value:

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) >= 10, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29143388
29143387


## Trip_distance

In [ ]:
variable_to_clean = "trip_distance"

It does not make sense that the trip distance is zero or negative, so remove those cases:

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) <= 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29030392


We also remove the lowest (below the 1% percentile) and highest tail (above the 99% percentile) of the distribution, since these values are way too low or too high, so they are probably incorrect.

In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+----+---+----+---+----+---+----+----+----+----+---+---+----+---+----+---+---+---+---+----+---+----+---+----+-----+-----------------+----+-----+-------+
|  %0| %1|  %2| %3|  %4| %5| %10| %15| %20| %25|%30|%35| %40|%45| %50|%55|%60|%65|%70| %75|%80| %85|%90| %95|  %96|              %97| %98|  %99|   %100|
+----+---+----+---+----+---+----+----+----+----+---+---+----+---+----+---+---+---+---+----+---+----+---+----+-----+-----------------+----+-----+-------+
|0.01|0.3|0.37|0.4|0.46|0.5|0.62|0.76|0.88|0.99|1.1|1.2|1.32|1.5|1.62|1.8|2.0|2.3|2.6|3.04|3.7|4.75|6.8|10.8|11.95|14.60269999999553|17.3|18.93|9496.98|
+----+---+----+---+----+---+----+----+----+----+---+---+----+---+----+---+---+---+---+----+---+----+---+----+-----+-----------------+----+-----+-------+



In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.01)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) > nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.99)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29030392
28463613


In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+----+---+----+---+---+----+----+---+---+---+---+----+---+----+---+---+----+---+---+---+----+----+----+----+-----+-----+-----+-----+
| %0|  %1| %2|  %3| %4| %5| %10| %15|%20|%25|%30|%35| %40|%45| %50|%55|%60| %65|%70|%75|%80| %85| %90| %95| %96|  %97|  %98|  %99| %100|
+---+----+---+----+---+---+----+----+---+---+---+---+----+---+----+---+---+----+---+---+---+----+----+----+----+-----+-----+-----+-----+
|0.3|0.36|0.4|0.45|0.5|0.5|0.66|0.78|0.9|1.0|1.1|1.2|1.33|1.5|1.62|1.8|2.0|2.27|2.6|3.0|3.6|4.57|6.32|10.0|10.9|12.06|14.81|17.31|18.97|
+---+----+---+----+---+---+----+----+---+---+---+---+----+---+----+---+---+----+---+---+---+----+----+----+----+-----+-----+-----+-----+



## Fare amount

This is the time-and-distance fare calculated by the meter. It does not make sense from a logical point of view that it is zero or lower. Also, even for Payment_type = 3 (No charge), we can see that the fare amount is generally higher than zero. Therefore, we can delete all values that are zero or lower.

In [ ]:
variable_to_clean = "fare_amount"

In [ ]:
nyc_yellow_tripdata_clean.groupby('Payment_type').agg(F.expr('count('+variable_to_clean+')'),
                                                      F.expr('min('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.25)'),
                                                      F.expr('mean('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.75)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.80)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.85)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.90)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.95)'),
                                                      F.expr('max('+variable_to_clean+')')      
                                                      ).show()

+------------+------------------+----------------+-----------------------------+------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------+
|Payment_type|count(fare_amount)|min(fare_amount)|percentile(fare_amount, 0.25)| mean(fare_amount)|percentile(fare_amount, 0.75)|percentile(fare_amount, 0.80)|percentile(fare_amount, 0.85)|percentile(fare_amount, 0.90)|percentile(fare_amount, 0.95)|max(fare_amount)|
+------------+------------------+----------------+-----------------------------+------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+-----------------------------+----------------+
|           1|          19832844|           -52.0|                          7.0|13.565015758203929|                         15.5|                         17.5|                         21.0|          

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) <= 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29173177


We also remove the lowest (below the 1% percentile) and highest tail (above the 99% percentile) of the distribution, since these values are way too low or too high, so they are probably incorrect.

In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|  %0| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40|%45|%50| %55| %60| %65| %70| %75| %80| %85| %90| %95| %96| %97| %98| %99|     %100|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+
|0.01|3.5|3.5|4.0|4.0|4.5|5.0|5.5|6.0|6.5|7.0|7.5|8.0|9.0|9.5|10.0|11.0|12.0|13.0|14.5|17.0|20.0|25.5|37.0|41.5|51.0|52.0|52.0|630461.82|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+---------+



In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.01)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) > nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.99)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29173177
28717024


In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| %0| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40|%45|%50| %55| %60| %65| %70| %75| %80| %85| %90| %95| %96| %97| %98| %99|%100|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|3.5|3.5|4.0|4.0|4.5|4.5|5.0|5.5|6.0|6.5|7.0|7.5|8.0|9.0|9.5|10.0|11.0|12.0|13.0|14.5|16.5|19.5|24.5|35.0|38.0|43.5|52.0|52.0|52.0|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+



## Extra

This value represents miscellaneous extras and surcharges. It should not contain negative values, so we remove those.

In [ ]:
variable_to_clean = "extra"

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29187611


According to the data dictionary, this variable should only include the 0.50 and 1 dollar rush hour and overnight charges, so it shouldn't be higher than 1 in any case. We remove values higher than one.

In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
| %0| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40|%45|%50|%55|%60|%65|%70|%75|%80|%85|%90|%95|%96|%97|%98|%99|%100|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.5|0.5|0.5|0.5|0.5|0.5|1.0|1.0|1.0|1.0|1.0|1.0|1.0|69.8|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+



In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) > 1, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29187611
29066237


We can see that there are some values that are not 0, 0.5 or 1. According to the data dictionary, this should never be the case, so we remove the values that do not meet this condition.

In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().sort(F.desc("count")).show()

+-----+--------+
|extra|   count|
+-----+--------+
|  0.0|15274428|
|  0.5| 9169244|
|  1.0| 4621755|
| null|  128363|
|  0.3|     469|
|  0.8|     247|
| 0.02|      30|
|  0.1|      16|
|  0.2|      14|
| 0.25|      12|
|  0.7|       4|
|  0.9|       3|
|  0.6|       3|
|  0.4|       3|
| 0.01|       2|
| 0.21|       1|
| 0.95|       1|
| 0.49|       1|
| 0.74|       1|
| 0.06|       1|
+-----+--------+
only showing top 20 rows



In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(~F.col(variable_to_clean).isin([0.0,0.5,1.0]), F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29066237
29065427


In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().orderBy(variable_to_clean).show()

+-----+--------+
|extra|   count|
+-----+--------+
| null|  129173|
|  0.0|15274428|
|  0.5| 9169244|
|  1.0| 4621755|
+-----+--------+



## MTA_tax

According to the data dictionary, the MTA tax is always 0.5 dollars and it is automatically triggered based on the metered rate in use. It should not contain negative values, so we can remove those.

In [ ]:
variable_to_clean = "mta_tax"

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29180980


In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
| %0| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40|%45|%50|%55|%60|%65|%70|%75|%80|%85|%90|%95|%96|%97|%98|%99| %100|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|0.0|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|0.5|54.51|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+



From the data distribution values we can see that (in line with what is specified in the data dictionary) almost all values are 0.5. Those that are not 0.5 are probably wrong, so we can substitute them with 0.5.

If all values are 0.5, this variable will not be of any use for a machine learning model, since it is not adding any relevant information. However, we can still leave it here in case that it helps to understand the values of other variables.

In [ ]:
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean, F.lit(0.5))

In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().sort(F.desc("count")).show()

+-------+--------+
|mta_tax|   count|
+-------+--------+
|    0.5|29194600|
+-------+--------+



## Tolls_amount

This is the tolls amount of all tolls paid in trip. Negative values do not make sense in here, so we remove them.

In [ ]:
variable_to_clean = "tolls_amount"

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29194527


We can see that the toll amount is zero in the majority of the cases, and 5.76 in many other cases. Very high values (such as the maximum of 1018.95) are most probably incorrect, so we can remove all values that are higher than the 99% percentile (5.76)

In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+-------+
| %0| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40|%45|%50|%55|%60|%65|%70|%75|%80|%85|%90| %95| %96| %97| %98| %99|   %100|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+-------+
|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|5.54|5.76|5.76|5.76|5.76|1018.95|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+-------+



In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) > nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.99)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194527
29095575


In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+
| %0| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40|%45|%50|%55|%60|%65|%70|%75|%80|%85|%90|%95| %96| %97| %98| %99|%100|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+
|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|5.54|5.76|5.76|5.76|5.76|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+



## Total_amount

The total amount charged to passengers (excluding cash tips) shouldn't be zero or lower. Also, even for Payment_type = 3 (No charge), we can see that the total amount is generally higher than zero. Therefore, we can delete all values that are zero or lower.

In [ ]:
variable_to_clean = "total_amount"

In [ ]:
nyc_yellow_tripdata_clean.groupby('Payment_type').agg(F.expr('count('+variable_to_clean+')'),
                                                      F.expr('min('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.25)'),
                                                      F.expr('mean('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.75)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.80)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.85)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.90)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.95)'),
                                                      F.expr('max('+variable_to_clean+')')      
                                                      ).show()

+------------+-------------------+-----------------+------------------------------+------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+-----------------+
|Payment_type|count(total_amount)|min(total_amount)|percentile(total_amount, 0.25)|mean(total_amount)|percentile(total_amount, 0.75)|percentile(total_amount, 0.80)|percentile(total_amount, 0.85)|percentile(total_amount, 0.90)|percentile(total_amount, 0.95)|max(total_amount)|
+------------+-------------------+-----------------+------------------------------+------------------+------------------------------+------------------------------+------------------------------+------------------------------+------------------------------+-----------------+
|           1|           19832844|           -70.59|                          9.36|17.844090385130777|                         19.55|                         21.96|        

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) <= 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29177804


We also remove the lowest (below the 1% percentile) and highest tail (above the 99% percentile) of the distribution, since these values are way too low or too high, so they are probably incorrect.

In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+----+----+---+---+----+---+---+---+----+----+---+---+-----+-----+----+----+----+----+----+----+----+----+-----+----+----+-----+-----+-----+---------+
|  %0|  %1| %2| %3|  %4| %5|%10|%15| %20| %25|%30|%35|  %40|  %45| %50| %55| %60| %65| %70| %75| %80| %85|  %90| %95| %96|  %97|  %98|  %99|     %100|
+----+----+---+---+----+---+---+---+----+----+---+---+-----+-----+----+----+----+----+----+----+----+----+-----+----+----+-----+-----+-----+---------+
|0.05|4.55|5.0|5.3|5.76|5.8|6.8|7.3|7.88|8.75|9.3|9.8|10.55|11.16|11.8|12.8|13.8|14.8|16.3|17.8|20.3|23.8|30.41|47.3|52.8|58.56|63.44|70.27|630463.12|
+----+----+---+---+----+---+---+---+----+----+---+---+-----+-----+----+----+----+----+----+----+----+----+-----+----+----+-----+-----+-----+---------+



In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.01)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) > nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.99)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29177804
28622077


In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+----+----+---+----+---+----+---+----+----+----+---+---+-----+-----+----+----+----+----+----+----+-----+----+-----+-----+-----+----+-----+-----+-----+
|  %0|  %1| %2|  %3| %4|  %5|%10| %15| %20| %25|%30|%35|  %40|  %45| %50| %55| %60| %65| %70| %75|  %80| %85|  %90|  %95|  %96| %97|  %98|  %99| %100|
+----+----+---+----+---+----+---+----+----+----+---+---+-----+-----+----+----+----+----+----+----+-----+----+-----+-----+-----+----+-----+-----+-----+
|4.55|4.94|5.3|5.76|5.8|5.94|6.8|7.48|8.15|8.76|9.3|9.8|10.55|11.16|11.8|12.8|13.8|14.8|16.0|17.8|20.15|23.3|29.16|43.86|48.07|52.8|58.56|64.56|70.27|
+----+----+---+----+---+----+---+----+----+----+---+---+-----+-----+----+----+----+----+----+----+-----+----+-----+-----+-----+----+-----+-----+-----+



## Improvement_surcharge

According to the data dictionary, this is a 0.30 dollar improvement surcharge that began being levied in 2015.

In [ ]:
variable_to_clean = "improvement_surcharge"

In [ ]:
show_quantiles(nyc_yellow_tripdata_clean, variable_to_clean).show()

+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|  %0| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40|%45|%50|%55|%60|%65|%70|%75|%80|%85|%90|%95|%96|%97|%98|%99|%100|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+
|-0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3|0.3| 1.0|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+



From the data distribution values we can see that (in line with what is specified in the data dictionary) almost all values are 0.3. Those that are not 0.3 are probably wrong, so we can substitute them with 0.3.

If all values are 0.3, this variable will not be of any use for a machine learning model, since it is not adding any relevant information. However, we can still leave it here in case that it helps to understand the values of other variables.

In [ ]:
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean, F.lit(0.3)) #Even though it is useless for a model, we keep it just in case it helps to understand other values

In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().orderBy(variable_to_clean).show()

+---------------------+--------+
|improvement_surcharge|   count|
+---------------------+--------+
|                  0.3|29194600|
+---------------------+--------+



## [TARGET!] Tip_amount

This is the variable that we want to predict with the machine learning model, so it is important to be extremely careful when cleaning it. According to the data dictionary, this field is automatically populated for credit card
tips. Cash tips are not included.

In [ ]:
variable_to_clean = "tip_amount"

Firstly, it does not make sense that this variable contains negative values, so we remove those.

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) < 0, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29194404


We can observe in the data distribution below that this variable contains many zeros, and it is generally in the range of [0-20] dollars. There are also some very high values (e.g., 450 dollars) that are most probably incorrect.

In [ ]:
show_quantiles_with_tail_detail(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+----+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+---+----+----+----+----+----+----+----+---+----+-----+-----+-----+-----+
| %0|%0.1|%0.5| %1| %2| %3| %4| %5|%10|%15|%20|%25|%30|%35|%40| %45| %50| %55| %60|%65| %70| %75| %80| %85| %90| %95| %96|%97| %98|  %99|%99.5|%99.9| %100|
+---+----+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+---+----+----+----+----+----+----+----+---+----+-----+-----+-----+-----+
|0.0| 0.0| 0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.1|1.0|1.15|1.36|1.56|1.76|2.0|2.16|2.46|2.85|3.35|4.25|6.55|7.51|8.7|10.2|11.71|14.32| 20.0|450.0|
+---+----+----+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+---+----+----+----+----+----+----+----+---+----+-----+-----+-----+-----+



Since the data dictionary mentions that this variable does not include cash tips, we will firstly analyze how it behaves according to the payment type.

The data dictionary indicates that:

*   Payment_type = 1 (Credit card)
*   Payment_type = 2 (Cash)
*   Payment_type = 3 (No charge)
*   Payment_type = 4 (Dispute)
*   Payment_type = 5 (Unknown)
*   Payment_type = 6 (Voided trip)

For credit card payments, the most common payment option in the data, the tip amount distribution seems logical (except for the high outliers). This is in line with what is specified in the data dictionary: "the tip amount field is automatically populated for credit card tips".

On the contrary, the tip amount is almost always zero for cash payments (there are only some values higher than zero on the very end of the distribution). This is also in line with what is specified in the data dictionary for the variable tip_amount: "Cash tips are not included.". We could try to extract the tip amount information from another variable in this case, but the only reasonable candidate seems to be the variable total_amount, and it is also specified in the data dictionary that it does not include cash tips.

For the other payment types, which are much more less common than credit card/cash, the tip_amount is also zero for a very large part of the distribution. That type of distribution, with so many zeroes, is completely different from the one corresponding to credit card payments, which is the only one that we can trust to be automatically populated according to the data dictionary. Therefore, it seems wise to only mantain the tip_amount values for credit card payments, since apparently that's the only payment option for which the tip amount is being properly collected.

There are almost 20 million credit card payments, so it is not  a problem to mantain only those to train the machine learning model. The more input variables that the model has, the more examples that we need to train a machine learning model (because the dimensionality gets higher), but 20 million rows are more than enough to train a machine learning model with this number of input variables.

In [ ]:
nyc_yellow_tripdata_clean.groupby('Payment_type').agg(F.expr('count('+variable_to_clean+')'),
                                                      F.expr('min('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.25)'),
                                                      F.expr('mean('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.75)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.80)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.85)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.90)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.95)'),
                                                      F.expr('max('+variable_to_clean+')')      
                                                      ).show()

+------------+-----------------+---------------+----------------------------+--------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+---------------+
|Payment_type|count(tip_amount)|min(tip_amount)|percentile(tip_amount, 0.25)|    mean(tip_amount)|percentile(tip_amount, 0.75)|percentile(tip_amount, 0.80)|percentile(tip_amount, 0.85)|percentile(tip_amount, 0.90)|percentile(tip_amount, 0.95)|max(tip_amount)|
+------------+-----------------+---------------+----------------------------+--------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+---------------+
|           1|         19832840|            0.0|                        1.35|  2.7603250734640192|                        3.06|                        3.55|                         4.2|                        5.26|      

In [ ]:
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col('Payment_type') == 1, F.col(variable_to_clean)).otherwise(F.lit(None)))

In [ ]:
nyc_yellow_tripdata_clean.groupby('Payment_type').agg(F.expr('count('+variable_to_clean+')'),
                                                      F.expr('min('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.25)'),
                                                      F.expr('mean('+variable_to_clean+')'),
                                                      F.expr('percentile('+variable_to_clean+', 0.75)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.80)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.85)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.90)'),
                                                      F.expr('percentile('+variable_to_clean+', 0.95)'),
                                                      F.expr('max('+variable_to_clean+')')      
                                                      ).show()

+------------+-----------------+---------------+----------------------------+------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+---------------+
|Payment_type|count(tip_amount)|min(tip_amount)|percentile(tip_amount, 0.25)|  mean(tip_amount)|percentile(tip_amount, 0.75)|percentile(tip_amount, 0.80)|percentile(tip_amount, 0.85)|percentile(tip_amount, 0.90)|percentile(tip_amount, 0.95)|max(tip_amount)|
+------------+-----------------+---------------+----------------------------+------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+---------------+
|           1|         19832840|            0.0|                        1.35|2.7603250734640192|                        3.06|                        3.55|                         4.2|                        5.26|              

We also remove the values at the very high tail of the distribution (above the 99.9% percentile), since those seem unreasonably high: 

In [ ]:
show_quantiles_with_tail_detail(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+----+----+---+---+---+---+---+---+---+----+----+---+---+----+----+---+----+----+----+----+----+----+---+----+---+----+-----+-----+-----+-----+-----+-----+
| %0|%0.1|%0.5| %1| %2| %3| %4| %5|%10|%15| %20| %25|%30|%35| %40| %45|%50| %55| %60| %65| %70| %75| %80|%85| %90|%95| %96|  %97|  %98|  %99|%99.5|%99.9| %100|
+---+----+----+---+---+---+---+---+---+---+----+----+---+---+----+----+---+----+----+----+----+----+----+---+----+---+----+-----+-----+-----+-----+-----+-----+
|0.0| 0.0| 0.0|0.0|0.0|0.0|0.0|0.6|1.0|1.0|1.16|1.35|1.5|1.6|1.76|1.95|2.0|2.15|2.32|2.55|2.82|3.06|3.55|4.2|5.26|8.1|9.08|10.08|11.67|12.66| 15.0|21.86|450.0|
+---+----+----+---+---+---+---+---+---+---+----+----+---+---+----+----+---+----+----+----+----+----+----+---+----+---+----+-----+-----+-----+-----+-----+-----+



In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) > nyc_yellow_tripdata_clean.agg(F.expr('percentile('+variable_to_clean+', 0.999)')).collect()[0][0], F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

19832840
19813010


In [ ]:
show_quantiles_with_tail_detail(nyc_yellow_tripdata_clean, variable_to_clean).show()

+---+----+----+---+---+---+---+---+---+---+----+----+---+----+----+----+---+----+----+----+---+----+----+----+---+----+---+----+-----+----+-----+-----+-----+
| %0|%0.1|%0.5| %1| %2| %3| %4| %5|%10|%15| %20| %25|%30| %35| %40| %45|%50| %55| %60| %65|%70| %75| %80| %85|%90| %95|%96| %97|  %98| %99|%99.5|%99.9| %100|
+---+----+----+---+---+---+---+---+---+---+----+----+---+----+----+----+---+----+----+----+---+----+----+----+---+----+---+----+-----+----+-----+-----+-----+
|0.0| 0.0| 0.0|0.0|0.0|0.0|0.0|0.6|1.0|1.0|1.16|1.35|1.5|1.59|1.76|1.95|2.0|2.15|2.32|2.55|2.8|3.06|3.55|4.16|5.2|8.01|9.0|10.0|11.65|12.6|14.64|18.55|21.86|
+---+----+----+---+---+---+---+---+---+---+----+----+---+----+----+----+---+----+----+----+---+----+----+----+---+----+---+----+-----+----+-----+-----+-----+



## PULocationID & DOLocationID

These two variables are TLC taxi zone IDs, so we leave them as they are to be able to use them to obtain the corresponding geometries if any kind of spatial analysis is necessary.

In [ ]:
nyc_yellow_tripdata_clean.groupby('PULocationID').count().sort(F.desc("count")).show()

+------------+-------+
|PULocationID|  count|
+------------+-------+
|         237|1158300|
|         161|1096421|
|         236|1052462|
|         186|1009238|
|         162| 998449|
|         230| 987406|
|         234| 945678|
|         170| 936841|
|          48| 915850|
|         142| 854307|
|          79| 843880|
|         163| 793569|
|         138| 783563|
|         239| 752892|
|         164| 695593|
|         107| 693345|
|         132| 678024|
|          68| 676911|
|         141| 665129|
|         249| 598680|
+------------+-------+
only showing top 20 rows



In [ ]:
nyc_yellow_tripdata_clean.groupby('DOLocationID').count().sort(F.desc("count")).show()

+------------+-------+
|DOLocationID|  count|
+------------+-------+
|         236|1090321|
|         161|1082368|
|         237|1016633|
|         170| 933700|
|         230| 901857|
|         162| 885057|
|         234| 812171|
|         186| 781972|
|          48| 780705|
|         142| 759280|
|         239| 718019|
|         163| 708679|
|          79| 696776|
|         141| 671440|
|          68| 644374|
|         164| 618584|
|         107| 613887|
|         238| 587767|
|         263| 523367|
|         246| 522124|
+------------+-------+
only showing top 20 rows



In [ ]:
nyc_yellow_tripdata_clean.groupby('PULocationID','DOLocationID').count().sort(F.desc("count")).show()

+------------+------------+------+
|PULocationID|DOLocationID| count|
+------------+------------+------+
|         264|         264|388461|
|         237|         236|164079|
|         236|         236|140366|
|         236|         237|134068|
|         237|         237|127857|
|         239|         238| 80936|
|         239|         142| 78684|
|         142|         239| 74788|
|         237|         162| 73882|
|         237|         161| 70201|
|         238|         239| 70175|
|         161|         237| 67663|
|         186|         230| 66629|
|         141|         236| 64164|
|         230|         186| 63875|
|         186|         161| 63236|
|          79|          79| 61792|
|          48|          48| 61330|
|         107|         170| 59693|
|         234|         170| 58952|
+------------+------------+------+
only showing top 20 rows



##RatecodeID

The variable RatecodeID indicates the final rate code in effect at the end of the trip. According to the data dictionary, the following are the possible values:

*   1= Standard rate
*   2=JFK
*   3=Newark
*   4=Nassau or Westchester
*   5=Negotiated fare
*   6=Group ride




In [ ]:
variable_to_clean = "RatecodeID"

In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().sort(F.desc("count")).show()

+----------+--------+
|RatecodeID|   count|
+----------+--------+
|         1|28370866|
|         2|  650659|
|         5|   95612|
|         3|   61733|
|         4|   15456|
|         6|     229|
|        99|      45|
+----------+--------+



We can see above that there are 45 rows with RatecodeID == 99. This rate code is not specified in the data dictionary, so we remove the RatecodeID for those rows:

In [ ]:
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())
nyc_yellow_tripdata_clean = nyc_yellow_tripdata_clean.withColumn(variable_to_clean,F.when(F.col(variable_to_clean) == 99, F.lit(None)).otherwise(F.col(variable_to_clean)))
print(nyc_yellow_tripdata_clean.filter(F.col(variable_to_clean).isNotNull()).count())

29194600
29194555


In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().sort(F.desc("count")).show()

+----------+--------+
|RatecodeID|   count|
+----------+--------+
|         1|28370866|
|         2|  650659|
|         5|   95612|
|         3|   61733|
|         4|   15456|
|         6|     229|
|      null|      45|
+----------+--------+



##Payment_type

According to the data dictionary, we can expect the following payment types:

*   1= Credit card
*   2= Cash
*   3= No charge
*   4= Dispute
*   5= Unknown
*   6= Voided trip

In [ ]:
variable_to_clean = "payment_type"

The values of this variable are in line with what is specified in the data dictionary, so we can leave them as they are.

In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().sort(F.desc("count")).show()

+------------+--------+
|payment_type|   count|
+------------+--------+
|           1|19832844|
|           2| 9164189|
|           3|  153885|
|           4|   43681|
|           5|       1|
+------------+--------+



##VendorID

According to the data dictionary, we can expect the following payment types:

* 1= Creative Mobile Technologies, LLC; 
* 2= VeriFone Inc

In [ ]:
variable_to_clean = "VendorID"

The values of this variable are in line with what is specified in the data dictionary, so we can leave them as they are.

In [ ]:
nyc_yellow_tripdata_clean.groupby(variable_to_clean).count().sort(F.desc("count")).show()

+--------+--------+
|VendorID|   count|
+--------+--------+
|       2|15951269|
|       1|13243331|
+--------+--------+



# Check clean result and save final dataframe

After our preprocessing, all variables in the dataset are of numerical types (except for the timestamp variables, which were already cleaned at the beginning of the process):

In [ ]:
nyc_yellow_tripdata_clean.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: integer (nullable = false)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = false)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = false)
 |-- total_amount: double (nullable = true)
 |-- meter_engagement_time_minutes: double (nullable = true)



As a final validation step before saving the clean dataset, we check the basic statistics of the numerical variables.

In [ ]:
nyc_yellow_tripdata_clean.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+--------+-----------------+------------------+---------------------+------------------+-----------------------------+
|summary|          VendorID|   passenger_count|     trip_distance|        RatecodeID|  store_and_fwd_flag|      PULocationID|      DOLocationID|       payment_type|       fare_amount|             extra| mta_tax|       tip_amount|      tolls_amount|improvement_surcharge|      total_amount|meter_engagement_time_minutes|
+-------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+-------------------+------------------+------------------+--------+-----------------+------------------+---------------------+------------------+-----------------------------+
|  count|          29194600|          29

All of the variables are contained within reasonable ranges now, so we can save the clean dataframe:

In [ ]:
nyc_yellow_tripdata_clean.write.mode('overwrite').parquet(work_folder+"yellow_tripdata_clean.parquet")